In [ ]:
# dependencies
!pip install boto3>=1.28.59
!pip install numpy
!pip install anthropic
!pip install mistralai
!pip install flask

!pip install langchain
!pip install langchainhub
!pip install langchain_community
!pip install langchain-aws

In [ ]:
!pip freeze 

In [ ]:
# global imports
import json
import os
import sys
import logging
import boto3
import botocore

logger = logging.getLogger(__name__)

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

# chat = ChatBedrock(
#     model_id="anthropic.claude-3-sonnet-20240229-v1:0",
#     model_kwargs={"temperature": 0.8, "max_tokens": 2048},
# )
chat = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    model_kwargs={"temperature": 0.8, "max_tokens": 2048},
)
# chat = ChatBedrock(
#     model_id="meta.llama3-8b-instruct-v1:0",
#     model_kwargs={"temperature": 0.8, "max_gen_len": 2048},
# )

messages = [
    HumanMessage(
        content="中国的首都在哪里？"
    )
]
#chat.invoke(messages)

for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)



In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage

chat = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={"temperature": 0.1},
)

messages = [
    HumanMessage(
        content="Translate this sentence from English to Chinese. I love programming."
    )
]
msg=chat(messages)
print(msg.content)

In [ ]:
werewolf_rule_v1 = """
1. 游戏分坏人(狼人)和好人(村民+预言家)两大阵营,他们的目标为:
- 坏人阵营只有狼人
- 好人阵营有村民和预言家
- 坏人阵营:消灭所有好人, 或者保证坏人数目大于好人数目
- 好人阵营:消灭所有坏人, 或者保证好人数目大于坏人数目

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼统一投票杀死一名玩家
- 晚上预言家只能查验一名玩家身份
- 晚上普通村民无法行动
- 白天分为讨论和投票两环节
- 白天在讨论环节，每个玩家必须参与讨论
- 白天在投票环节，每个玩家必须投票或者放弃

"""

werewolf_command_v1 = """
- WolfVote: 夜晚投票(狼人专属行动),参数: target=存活玩家
- ProphetCheck: 夜晚查验(预言家专属行动), 参数: target=存活玩家
- PlayerDoubt: 白天怀疑(所有玩家白天可选行动, 非投票), 参数: target=存活玩家 
- PlayerVote: 白天投票, 参数: target=存活玩家 
- Debate: 白天讨论, 参数: content=思考/理由 
- GetAllPlayersName: 玩家信息, 参数: 无 
- DeathWords: 死亡遗言, 参数: content=给予玩家线索
- Pass: 玩家弃权参数: 无 
"""

template_player_role = """你是资深的社交游戏玩家, 熟悉《狼人杀》游戏规则:
<game_rules>
{game_rule}
</game_rules>

你熟悉该游戏所有命令:
<commands>
{commands}
</commands>

<reflections>
- 按照游戏规则，第一个夜晚死亡的一定是村民或者预言家，狼人没必要第一晚上自杀
- 看完P3玩家昨天白天投票，明显感觉他在混淆是非，很有可能在给狼人分票
- 平民玩家由于信息缺失，所以狼人要尽量引导他们去集火其他人
- 第一个夜晚所有的行动都是随机的
- 作为第一个死亡的玩家，其实信息有限，我就靠第六感推理...
</reflections>

<references>
- {{"action": "Pass"}}
- {{"action": "WolfVote", "target": "小明"}}
- {{"action": "ProphetCheck", "target": "P1"}}
- {{"action": "PlayerVote", "target": "老王"}}
- {{"action": "Debate", "content": "我的推理为xx是狼，原因是..."}}
- {{"action": "DeathWords", "content": "我觉得xx有很大的嫌疑, 原因是..."}}
</references>

历史信息:
<history>
{history}
</history>

记住，你支持的玩家是 {nickname}, 身份是 {role}, 性格为 {character}, 必须帮助玩家进行这个游戏
接下来你的目的是: 通过一步一步思考决策引导游戏往有利于的方向进行, 最终赢得比赛. 

决策满足下面要求:
- 内容不要罗嗦, 不要超过50字数限制,少讲废话, 突出重点
- 判断场上信息真伪, 运用辩解,对抗,欺骗,伪装,坦白等等任意策略来做决策
- 决策分为两类:思考或行动
- 思考:逐步思考,判断信息真伪,分析游戏形势等等,参考 <reflections> 选择合适的输出
- 行动:参考<references>按照json字符串格式输出,必须包含action key, action必须在<commands>中选择


{input}
""".replace("{game_rule}", werewolf_rule_v1).replace("{commands}", werewolf_command_v1)

print(template_player_role)

In [ ]:
from shared.LangchainMini.LangchainMini import LangchainMini, LangchainMiniMemory, LangchainMiniPromptTemplate

# A和C是同学,确认说"ok"
# B和C是亲戚,确认说"ok"
# A和C会是什么关系,开始推理

# template = LangchainMiniPromptTemplate('''历史记录:{history}
# B和C是什么关系，开始推理"
# ''')

#template = LangchainMiniPromptTemplate(template_player_role)
#prompt = template.format(input="你是狼人，第一个夜晚如何行动?")

model_id="mistral.mixtral-8x7b-instruct-v0:1"
model_id="meta.llama3-8b-instruct-v1:0"
#model_id="anthropic.claude-3-sonnet-20240229-v1:0"

agent = LangchainMini(model_id=model_id, stream=True)
memory = LangchainMiniMemory(k=1, llm=agent)
llm = LangchainMini(model_id=model_id, stream=True, memory=memory)
# llm.System("Answer the question in Japanese.")

templates = ['{history} A和C是同学,确认说"ok"', '{history} B和C是亲戚,确认说"ok"', '{history} A和C会是什么关系,开始推理']
templates = ['{history} A and C are classmates, confirmed with "ok"', 
             '{history} B and C are relatives,confirmed with "ok"', 
             '{history} What relation between A and C, let us think step by step!']
for template in templates:
    _template = LangchainMiniPromptTemplate(template)
    prompt = _template.format()
    llm.invoke(prompt)

In [ ]:
from shared.GameAssistant import GameAssistant
from shared.PeTemplates import *

#ga = GameAssistant(template_assistant_role, None)

#ga.DoAnswer("总结下内容")

# print(werewolf_command_v1)

# print(template_wolf_role)

# print(template_prophet_role)

# print(template_player_role)

# LoadPlayerPrompts()

# print(roles_dict)

def sort_players_by_role(players):
    # 定义角色优先级字典
    role_priorities = {
        "狼人": 1,
        "预言家": 2,
        "女巫": 3,
        "村民": 4
    }

    # 根据角色优先级对玩家列表进行排序
    sorted_players = sorted(players, key=lambda player: role_priorities.get(player["身份"], 5))

    return sorted_players

players = [
    {"名字": "张三", "身份": "狼人"},
    {"名字": "李四", "身份": "村民"},
    {"名字": "王五", "身份": "预言家"},
    {"名字": "赵六", "身份": "女巫"},
    {"名字": "钱七", "身份": "村民"}
]

sorted_players = sort_players_by_role(players)
print(sorted_players)

In [ ]:
from shared.GameServer import GameServer

GS = GameServer()
GS.Run("0.0.0.0", 8000)


In [ ]:
from shared.GamePlayer import GamePlayer
from shared.GameMaster import GameMaster
from shared.PeTemplates import *
LoadPlayerPrompts()
GM = GameMaster(1, False)
for player in roles_dict["players"]:
    _player = GamePlayer(player, GM)
    
    _player.DoPlanning(game_config_dict["player"]["action_plan_night"], 1)


In [ ]:
def get_value_from_object(obj, key_path):
    keys = key_path.split(".")
    value = obj
    for key in keys:
        if isinstance(value, dict):
            value = value.get(key)
            if value is None:
                break
        else:
            raise TypeError(f"Cannot access key '{key}' from non-dictionary object")
    return value

# 使用示例
data = {"a": {"b": 2}}
key_path = "a.b"
value = get_value_from_object(data, key_path)
print(value)  # 输出: 42

In [ ]:
def ConvertToJson(data):
    try:
        if isinstance(data, str):
            return json.loads(data)
        else:
            return data
    except json.JSONDecodeError:
        print("Invalid JSON string")
        return data
    
ABC = ['{"action": "Debate", "agent": "Aiden", "content": "大家有没有注意到,Ethan今天的言行似乎有些反常?他先是对Mason的身份表示怀疑,但随后又为Mason辩护,甚至提出了一些自相矛盾的论点。我觉得Ethan的这种反复无常的态度值得我们深思。"}']
ABC = [ConvertToJson(ABC[len(ABC)-1])]
print(ABC[len(ABC)-1])
print(ABC[len(ABC)-1]["content"])

In [1]:
from shared.GameMaster import GameMaster
from shared.PeTemplates import *

GM = GameMaster(num=2, queueSize=50, quick=False)
GM.ResetGame("en")
GM.RunGame()
GM.EndGame()

05/24/2024 12:13:21 PM NAME: Adam ROLE: witch GENER: 1
05/24/2024 12:13:21 PM 	 MODEL: anthropic.claude-3-sonnet-20240229-v1:0, MAX_TOKENS:2048, TEMP:0.8
05/24/2024 12:13:21 PM 	 MODEL: anthropic.claude-3-sonnet-20240229-v1:0, MAX_TOKENS:2048, TEMP:0.8
05/24/2024 12:13:21 PM 	 MODEL: mistral.mixtral-8x7b-instruct-v0:1, MAX_TOKENS:2048, TEMP:0.8
05/24/2024 12:13:21 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
05/24/2024 12:13:21 PM NAME: Bella ROLE: wolf GENER: 0
05/24/2024 12:13:21 PM 	 MODEL: anthropic.claude-3-sonnet-20240229-v1:0, MAX_TOKENS:2048, TEMP:0.8
05/24/2024 12:13:21 PM 	 MODEL: anthropic.claude-3-sonnet-20240229-v1:0, MAX_TOKENS:2048, TEMP:0.8
05/24/2024 12:13:21 PM 	 MODEL: meta.llama3-8b-instruct-v1:0, MAX_TOKENS:2048, TEMP:0.8
05/24/2024 12:13:21 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
05/24/2024 12:13:21 PM NAME: Charles ROLE: villager GENER: 1
05/24/2024 12:13:21 PM 	 MODEL: anthropic.claude-3-sonnet-20240229-

Observation: It is currently the night phase of Day 1. My identity is a werewolf, and my teammates are Bella and Gabriel. The remaining players' identities are unknown. There are 2 werewolves, 4 villagers, 1 prophet, and 1 witch in the game.

Reasoning: Since it is the first night, no information about other players' roles has been revealed yet. I should be cautious and try to gather more information before making any rash decisions. Targeting players with special roles like the prophet or witch too early could alert the villagers.

Thinking:
1) As an independent thinker, I should not blindly follow my teammates' suggestions if they seem illogical or reckless.
2) However, it is crucial to coordinate with my werewolf teammate Gabriel to avoid exposing ourselves.
3) For now, the safest option may be to eliminate a seemingly ordinary villager to avoid arousing suspicion.
4) I should pay close attention to any hints or behavior during the day that could reveal the prophet's or witch's iden

05/24/2024 12:13:36 PM 	--------- TIME: NIGHT-1 ACTION: DoReflect ---------
05/24/2024 12:13:36 PM 		******** PALYER: Bella ROLE: wolf ********
05/24/2024 12:13:36 PM 		******** ACTOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:13:36 PM 		******** REVISOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:13:36 PM 		******** SUMMERY: meta.llama3-8b-instruct-v1:0********
05/24/2024 12:13:36 PM 	REFLECT QUESTION: Game progress: Current state:Adam:alive,Bella:alive,Charles:alive,Daphne:alive,Ethan:alive,Fiona:alive,Gabriel:alive,Haley:alive.. Player information: Remember, the player you support is Bella, identity wolf, personality Independent Thinker. Team: 2wolf ,4villager ,1prophet ,1witch . Teammates: Bella,Gabriel.. Please extract useful information, then think through step-by-step.. Time: NIGHT-1. Player decision: ["{\"action\": \"WolfVote\", \"agent\": \"Bella\", \"target\": \"Daphne\"}"].
05/24/2024 12:13:36 PM HTTP Request: POST https://bedrock-runti

Observation: It is the first night of the game. Bella, who is a werewolf, has voted to eliminate Daphne during the night phase. Her teammate Gabriel, who is also a werewolf, needs to coordinate and vote for the same target.

Scoring:

Impact of the decision on oneself (Bella):
3 - Favorable. As a werewolf, voting to eliminate a villager or a player with a special role is a good strategy to weaken the opposing team.

Impact of the decision on one's own team (Werewolves):
3 - Favorable. If both werewolves vote for the same target, they increase the chances of successfully eliminating that player, which benefits their team's chances of winning.

Score: {"score": 6}

05/24/2024 12:13:42 PM 		 DoReflect: ['{"score": 6}']
05/24/2024 12:13:42 PM 	--------- TIME: NIGHT-1 ACTION: DoValidate ---------
05/24/2024 12:13:42 PM 		******** PALYER: Bella ROLE: wolf ********
05/24/2024 12:13:42 PM 		******** ACTOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:13:42 PM 		******** REVISOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:13:42 PM 		******** SUMMERY: meta.llama3-8b-instruct-v1:0********
05/24/2024 12:13:42 PM 		 DoValidate: ['{"action": "WolfVote", "agent": "Bella", "target": "Daphne"}']
05/24/2024 12:13:42 PM 	--------- TIME: NIGHT-1 ACTION: DoAction ---------
05/24/2024 12:13:42 PM 		******** PALYER: Bella ROLE: wolf ********
05/24/2024 12:13:42 PM 		******** ACTOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:13:42 PM 		******** REVISOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:13:42 PM 		******** SUMMERY: meta.llama3-8b-instruct-v1:0********
05/24/2024 12:13:42 PM 
 SystemLog:

...

05/24/2024 12:15:50 PM 		 DoReflect: ['{"score": 7}']
05/24/2024 12:15:50 PM 	--------- TIME: DAY-2 ACTION: DoValidate ---------
05/24/2024 12:15:50 PM 		******** PALYER: Adam ROLE: witch ********
05/24/2024 12:15:50 PM 		******** ACTOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:15:50 PM 		******** REVISOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:15:50 PM 		******** SUMMERY: mistral.mixtral-8x7b-instruct-v0:1********
05/24/2024 12:15:50 PM 		 DoValidate: ['{\n"action": "Debate",\n"agent": "Adam",\n"content": "Daphne, can you share any insights about your elimination and revival? Any information could be helpful in identifying the werewolves among us. Additionally, I encourage everyone to speak openly and honestly, as we must work together to uncover the truth."\n}']
05/24/2024 12:15:50 PM 	--------- TIME: DAY-2 ACTION: DoAction ---------
05/24/2024 12:15:50 PM 		******** PALYER: Adam ROLE: witch ********
05/24/2024 12:15:50 PM 		******** ACTOR:

Based on the given information, here's a summary of what happened:

* At NIGHT-1, both Bella and Gabriel voted to eliminate Daphne, but due to the Witch's intervention, Daphne was revived.
* This is the second time Daphne was revived, indicating that Daphne is likely a witch or has a protective effect that prevents her from being eliminated.
* At DAY-2, Adam starts a debate, asking Daphne to share any insights about her elimination and revival. He also encourages everyone to speak openly and honestly to uncover the truth.

From this, we can deduce that:

* Daphne is likely a witch or has a protective effect that prevents her from being eliminated.
* The werewolves among the players are still unknown, and it's crucial to continue investigating and gathering information to identify them.
* The debate started by Adam may lead to valuable insights and potentially help reveal the werewolves.

05/24/2024 12:15:59 PM 	--------- TIME: DAY-2 ACTION: DoAnswer ---------
05/24/2024 12:15:59 PM 		******** PALYER: Bella ROLE: wolf ********
05/24/2024 12:15:59 PM 		******** ACTOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:15:59 PM 		******** REVISOR: anthropic.claude-3-sonnet-20240229-v1:0********
05/24/2024 12:15:59 PM 		******** SUMMERY: meta.llama3-8b-instruct-v1:0********
05/24/2024 12:15:59 PM 	ACTOR QUESTION: Current state:Adam:alive,Bella:alive,Charles:alive,Daphne:alive,Ethan:alive,Fiona:alive,Gabriel:alive,Haley:alive.Remember, the player you support is Bella, identity wolf, personality Independent Thinker. Team: 2wolf ,4villager ,1prophet ,1witch . Teammates: Bella,Gabriel.. Please extract useful information, then think through step-by-step. Time: Day 2 discussion phase. How should you act?
05/24/2024 12:15:59 PM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "H

Observation: It is currently the discussion phase of Day 2. My identity is a werewolf, and my teammate is Gabriel. Daphne has been revived twice, indicating she is likely the witch or has a protective effect. Adam has started a debate, encouraging everyone to share information openly.

Reasoning: Since Daphne's identity as the witch or having a protective effect is becoming more apparent, it would be unwise to target her directly for now. Doing so would risk exposing my identity as a werewolf. It is crucial to gather more information during this discussion phase to identify potential villagers or the prophet.

Thinking:
1) As an independent thinker, I should not blindly follow my teammate Gabriel's suggestions if they seem illogical or reckless.
2) However, it is still essential to coordinate with Gabriel to avoid exposing ourselves as werewolves.
3) During the discussion, I should carefully observe the behavior and statements of other players, looking for any inconsistencies or suspic

KeyboardInterrupt: 